<a href="https://colab.research.google.com/github/ryanvuu/csc-365-lab7/blob/main/csc_365_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mysql-connector-python pandas

In [ ]:
import getpass
db_password = getpass.getpass()
# print(db_password)

In [ ]:
import mysql.connector
conn = mysql.connector.connect(user='rvu03', password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='rvu03')

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM lab7_rooms")
result = cursor.fetchall()
print(result)